In [16]:
from jmetal.algorithm.singleobjective.genetic_algorithm import GeneticAlgorithm
from jmetal.algorithm.multiobjective.nsgaii import NSGAII
from jmetal.algorithm.singleobjective.local_search import LocalSearch
from jmetal.algorithm.singleobjective.simulated_annealing import SimulatedAnnealing
from jmetal.operator import BinaryTournamentSelection
from jmetal.operator.crossover import PMXCrossover, SBXCrossover, SPXCrossover, CXCrossover, DifferentialEvolutionCrossover
from jmetal.operator.mutation import PermutationSwapMutation, PolynomialMutation, NonUniformMutation, UniformMutation, SimpleRandomMutation
# from jmetal.problem.singleobjective.tsp import TSP
from jmetal.util.comparator import MultiComparator
from jmetal.util.density_estimator import CrowdingDistance
from jmetal.util.ranking import FastNonDominatedRanking
from jmetal.util.termination_criterion import StoppingByEvaluations
from jmetal.util.solution import get_non_dominated_solutions
from jmetal.core.problem import PermutationProblem
from jmetal.core.solution import PermutationSolution
import math
import random
import re
import tsplib95
import matplotlib.pyplot as plt

In [12]:
class TSP(PermutationProblem):
    """ Class representing TSP Problem. """

    def __init__(self, instance: str = None):
        super(TSP, self).__init__()
        distance_matrix, number_of_cities = self.__read_from_file(instance)
        self.distance_matrix = distance_matrix
        self.obj_directions = [self.MINIMIZE]
        self.number_of_variables = number_of_cities
        self.number_of_objectives = 1
        self.number_of_constraints = 0

 

    def __read_from_file(self, filename: str):
        if filename is None:
            raise FileNotFoundError('Filename can not be None')
        with open(filename) as file:
            lines = file.readlines()
            data = [line.lstrip() for line in lines if line != ""]
            dimension = re.compile(r'[^\d]+')
            for item in data:
                if item.startswith('DIMENSION'):
                    dimension = int(dimension.sub('', item))
                    break
            c = [-1.0] * (2 * dimension)
            for item in data:
                if item[0].isdigit():
                    j, city_a, city_b = [float(x.strip()) for x in item.split(' ')]
                    j = int(j)
                    c[2 * (j - 1)] = city_a
                    c[2 * (j - 1) + 1] = city_b
            matrix = [[-1] * dimension for _ in range(dimension)]
            for k in range(dimension):
                matrix[k][k] = 0
                for j in range(k + 1, dimension):
                    dist = math.sqrt((c[k * 2] - c[j * 2]) ** 2 + (c[k * 2 + 1] - c[j * 2 + 1]) ** 2)
#                     dist = round(dist)
                    matrix[k][j] = dist
                    matrix[j][k] = dist
            return matrix, dimension

    def evaluate(self, solution: PermutationSolution) -> PermutationSolution:
        fitness = 0
        global count
        count+=1
        for i in range(self.number_of_variables - 1):
            x = solution.variables[i]
            y = solution.variables[i + 1]
            fitness += self.distance_matrix[x][y]
        first_city, last_city = solution.variables[0], solution.variables[-1]
        fitness += self.distance_matrix[first_city][last_city]
        solution.objectives[0] = fitness
        return solution

    def create_solution(self) -> PermutationSolution:
        new_solution = PermutationSolution(number_of_variables=self.number_of_variables,
                                           number_of_objectives=self.number_of_objectives)
        new_solution.variables = random.sample(range(self.number_of_variables), k=self.number_of_variables)
        return new_solution
    
    @property
    def number_of_cities(self):
        return self.number_of_variables

    def get_name(self):
        return 'Symmetric TSP'

In [15]:
count = 0
fitness = 0
data = tsplib95.load_problem('./Data/dj38.tsp')
problem = TSP(instance='./Data/dj38.tsp')

print('Cities: ', problem.number_of_variables)

algorithm = GeneticAlgorithm(
    problem=problem,
    population_size=50,
    offspring_population_size=50,
    mutation=PermutationSwapMutation(0.30),
    crossover=PMXCrossover(0.50),
    selection=BinaryTournamentSelection(
        MultiComparator([FastNonDominatedRanking.get_comparator(),
                         CrowdingDistance.get_comparator()])),
    termination_criterion=StoppingByEvaluations(max_evaluations=100000)
)
algorithm.run()
result = algorithm.get_result()

 

print('Algorithm: {}'.format(algorithm.get_name()))
print('Problem: {}'.format(problem.get_name()))
print('Solution: {}'.format(result.variables))
print('Fitness: {}'.format(result.objectives[0]))
print('Computing time: {}'.format(algorithm.total_computing_time))

C:\Users\sreekanth\Anaconda3\envs\sreekanth\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) load_problem. (Will be removed in newer versions. Use `tsplib95.load` instead.) -- Deprecated since version 7.0.0.
  """Entry point for launching an IPython kernel.


Cities:  38
Algorithm: Genetic algorithm
Problem: Symmetric TSP
Solution: [28, 20, 9, 13, 11, 10, 8, 7, 6, 3, 0, 1, 15, 16, 23, 21, 19, 22, 24, 25, 36, 37, 32, 33, 35, 30, 26, 18, 17, 27, 34, 31, 29, 5, 2, 4, 12, 14]
Fitness: 11221.559916432281
Computing time: 9.884312152862549
